In [27]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [28]:
with open('/content/intents.json') as file:
    data = json.load(file)

In [35]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

print("The Intent Data", data['intents'])
print()
print("Training Sentences: ", training_sentences)
print()
print("Labels: ", training_labels)
print()
print("Responses: ", responses)

The Intent Data [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hi there', 'Hello', 'Hey there', 'Howdy', 'Hola', 'Bonjour', 'Konnichiwa', 'Guten tag', 'Ola'], 'responses': ['Hello there. Tell me how are you feeling today?', 'Hi there. What brings you here today?', 'Hi there. How are you feeling today?', 'Great to see you. How do you feel currently?', "Hello there. Glad to see you're back. What's going on in your world right now?"]}, {'tag': 'morning', 'patterns': ['Good morning'], 'responses': ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]}, {'tag': 'afternoon', 'patterns': ['Good afternoon'], 'responses': ['Good afternoon. How is your day going?']}, {'tag': 'evening', 'patterns': ['Good evening'], 'responses': ['Good evening. How has your day been?']}, {'tag': 'night', 'patterns': ['Good night'], 'responses': ['Good night. Get some proper sleep', 'Good night. Sweet dreams.']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you late

In [37]:
print(num_classes)
print(len(training_labels))

80
232


In [38]:
lbl_encoder=LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels=lbl_encoder.transform(training_labels)

training_labels

array([44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 55,  1, 10, 58, 43,
       43, 43, 43, 43, 43, 43, 43, 73, 73, 73, 73, 73, 60, 57,  0,  0,  0,
        0,  0,  0,  0,  0, 67,  5,  5,  5, 56, 56, 56, 48, 48, 48, 48, 48,
       48, 48, 65, 65, 65, 65, 65, 65, 65, 65, 70, 70, 70, 70, 70, 78, 78,
       78, 78, 78,  8,  8,  8,  8, 45, 45, 45, 45, 45, 45, 45,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  2,  2, 61, 61, 61, 61, 68, 68, 68,
       68, 68, 68, 66, 66, 66, 66,  6,  6,  6,  6,  6,  6, 74, 74, 74, 74,
       74, 74,  9,  9,  9,  9,  9, 72, 72, 72, 72, 72, 47, 47, 47, 46, 46,
       46,  7,  7,  7,  7,  7,  7,  7,  7, 49, 49, 64, 64, 64, 79, 79, 79,
       79, 71, 71, 71, 71, 52, 52, 52, 69, 69, 69, 69, 42,  3, 63, 63, 59,
       59, 59, 51, 51, 51, 76, 76, 53, 53, 77, 77, 62, 75, 75, 75, 50, 50,
       50, 54, 54, 11, 11, 22, 22, 33, 33, 37, 37, 37, 37, 38, 38, 39, 39,
       39, 40, 41, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26,
       27, 28, 29, 29, 30

In [41]:
# Tokenization process (Vectorising the Data)

vocab_size = 1000 # the size of the vocabulary in the text data
embedding_dim = 16
max_len = 20
oov_token = "<OOV>" # Out Of Vocabulary are words not present in the vocabulary and will be mapped as UNK token

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=20)

In [42]:
# Training the Neural Network of our Chatbot with 3 Hidden layers

model = Sequential() # The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs.
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D()) # Pooling. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network
model.add(Dense(16, activation='relu')) # Activation function Relu 1
model.add(Dense(16, activation='relu')) # Activation function Relu 2
model.add(Dense(num_classes, activation='softmax')) # Activation function Softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # This loss function is mathematically same as the categorical_crossentropy.

print(model.summary())
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_9           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0173 - loss: 4.3819
Epoch 2/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0428 - loss: 4.3760  
Epoch 3/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0561 - loss: 4.3697  
Epoch 4/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0351 - loss: 4.3635  
Epoch 5/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0502 - loss: 4.3554  
Epoch 6/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0678 - loss: 4.3455
Epoch 7/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0336 - loss: 4.3316  
Epoch 8/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0322 - loss: 4.3084  
Epoch 9/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0415 - loss: 4.2799 
Epoch 10/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0244 - loss: 4.2461
Epoch 11/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0510 - loss: 4.1847 
Epoch 12/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0

In [43]:
import pickle
model.save('chat_model.h5')
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)
with open('lbl_encoder.pickle','wb') as ecn_file:
    pickle.dump(lbl_encoder,ecn_file,protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
with open('tokenizer.pickle','rb') as handle:
    tokenizer=pickle.load(handle)
with open('lbl_encoder.pickle','rb') as ecn_file:
    lbl_encoder=pickle.load(ecn_file)
model=keras.models.load_model('chat_model.h5')

In [45]:
def generate_response(user_input):
    # convert user input to a padded sequence of integers using the tokenizer
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, truncating='post', maxlen=20)

    # predict the class probabilities for the input sequence using the trained model
    predictions = model.predict(padded_sequence)

    # find the index of the class with the highest probability
    class_index = np.argmax(predictions)

    # convert the class index back to the corresponding tag using the label encoder
    tag = lbl_encoder.inverse_transform([class_index])[0]

    # choose a random response from the list of responses for the predicted tag
    responses = data['intents']
    for intent in responses:
        if intent['tag'] == tag:
            result = np.random.choice(intent['responses'])
            break

    return result

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
A therapist is a broad designation that refers to professionals who are trained to provide treatment and rehabilitation. The term is often applied to psychologists, but it can include others who provide a variety of services, including social workers, counselors, life coaches, and many others. 


In [47]:
def chat():
    print("Start Talking with the bot(type quit to stop!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = generate_response(inp)

        print("Bot: ", results)

In [48]:
chat()

Start Talking with the bot(type quit to stop!
You: Who created you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot:  I was created by >.
You: Who created you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot:  I was created by >.
You: What's the difference between sadness and depression?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot:  Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. Other ways to talk about sadness might be feeling low, feeling down, or feeling blue.A person may say they are feeling depressed, but if it goes away on its own and doesn't impact life in a big way, it probably isn't the illness of depression. Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names